## 10장. 심층신경망

### 1. 데이터를 준비하고 밀집층 만들기
**1.1 패션 MNIST 데이터셋 준비하기**

In [23]:
# 텐서플로 라이브러리에서 keras 패키지를 임포트한다.
from tensorflow import keras

# keras.datasets.fashion_mnist 모듈 아래 load_data()은 패션 MNIST 데이터를 로드하고
# 훈련 세트와 테스트 세트로 나누어 반환한다.
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

**1.2 데이터 전처리와 훈련세트 & 검증세트 나누기**

In [24]:
from sklearn.model_selection import train_test_split

# 각 픽셀값(0~255)을 255로 나누어 0~1 사이의 값으로 정규화한다.
train_scaled = train_input / 255.0

# reshape()으로 원본 배열(60000, 28, 28)의 두번째, 세 번째 차원을 1차원으로 한다.
train_scaled = train_scaled.reshape(-1, 28*28)

# 전처리된 데이터를 다시 훈련세트와 검증세트로 분리한다.
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

**1.3 밀집층 2개 만들기**

In [25]:
# 은닉층으로 사용될 밀집층 객체 dense1을 생성한다.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))

# 출력층으로 사용될 밀집층 객체 dense2을 생성한다.
dense2 = keras.layers.Dense(10, activation='softmax')

## 2. 심층 신경망 모델 만들기

In [26]:
# 밀집층 dense1, dense2 로 심층신경망모델을 생성한다.
model = keras.Sequential([dense1, dense2])

In [27]:
# 케라스는 모델의 summary()를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 3. 층을 추가하는 다른 방법

**3.1 Sequential 클래스의 생성자 안에서 층을 추가하는 방법**

In [28]:
model = keras.Sequential([
  keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
  keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [29]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**3.2 add()로 층을 추가하는 방법**

In [58]:
model = keras.Sequential(name="Fashion_MNIST_model")
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

In [59]:
model.summary()

Model: "Fashion_MNIST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**3.3 층을 추가해서 다시 훈련시키기**

In [60]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5635 - accuracy: 0.8081
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4088 - accuracy: 0.8518
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3725 - accuracy: 0.8643
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3508 - accuracy: 0.8733
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3344 - accuracy: 0.8793


### 4. 렐루 활성화 함수

**4.1 렐루함수와 Flatten 층을 추가해서 심층심경망 모델 만들기**

In [70]:
# 신경망 모델 객체를 생성한다.
model = keras.Sequential(name='Fashion_MNIST_model')

# 입력을 1차원으로 펼치는 Flatten 층을 추가한다.
model.add(keras.layers.Flatten(input_shape=(28, 28)))

# add()로 은닉층과 출력층을 추가한다.
# 은닉층의 활성화 함수에 렐루(relu)함수를 지정한다.
model.add(keras.layers.Dense(100, activation='relu', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

In [71]:
model.summary()

Model: "Fashion_MNIST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**4.2 훈련 데이터 준비하기(reshape() 사용 안함)**

In [72]:
(train_input, train_target), (test_input, test_target) = \
  keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

**4.3 다시 모델 설정하고 훈련시키기**

In [68]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5316 - accuracy: 0.8117
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3954 - accuracy: 0.8582
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3589 - accuracy: 0.8714
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3360 - accuracy: 0.8788
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3187 - accuracy: 0.8860


**4.4 검증세트로 모델 평가하기**

In [69]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3687 - accuracy: 0.8773


[0.3686736524105072, 0.8772500157356262]